In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
joined_files_paths = os.path.join('D:\pythonProject1\dds_ch2_nyt', "nyt*.csv")
joined_files_paths_list = glob.glob(joined_files_paths)
user_data = pd.concat(map(pd.read_csv,joined_files_paths_list),ignore_index=True)

Підгрузимо дані з гіта, зчитаємо та об'єднаємо в один фрейм. Виведемо

In [ ]:

user_data


Розділимо користувачів на вікові підгрупи 

In [ ]:
bins = [user_data['Age'].min(), 18, 25, 35, 45, 55, 65,user_data['Age'].max()]
labels = ["<18", "18-24", "25-34", "35-44", "45-54", "55-64", "65+"]


In [ ]:
user_data['age_group'] = pd.cut(user_data['Age'], bins=bins, labels=labels)
user_data

Відфільтруємо дані від невизначених груп

In [ ]:
filtered_data = user_data.dropna(subset=['age_group'])
filtered_data

Додамо ще одну колонку зі значенням CTR (відношення clicks до impressions)

In [ ]:
filtered_data['CTR'] = filtered_data['Clicks'] / filtered_data['Impressions']

Обчислимо  к-сть показів та середнє значення показника CTR при к-сті показів для кожної вікової групи

In [ ]:
age_group_data = filtered_data.groupby('age_group').agg({'Impressions': 'sum', 'CTR': 'mean'}).reset_index();
age_group_data

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='age_group', y='CTR', data=age_group_data)

plt.xlabel('Age Group')
plt.ylabel('CTR')
plt.title('Коефіцієнт кліків (CTR) за віковими групами')
plt.xticks(rotation=45)

plt.show()

З даного графіка можем побачити, що найбільший показник кліків щодо показів належить групі 65+, далі йде група <18 та 55-64. Це свідчить про те, що оголошення працюють не надто ефективно для людей віком 18-54 (показник у них майже однаковий).

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='age_group', y='Impressions', data=age_group_data)

plt.xlabel('Age Group')
plt.ylabel('Impressions')
plt.title('Покази за віковими групами')
plt.xticks(rotation=45)

plt.show()

Бачимо, що топ-3 к-сть показів у вікової групи 25-54. У зв'язку з низьким показником CTR для цих груп, можна зробити висновок, що оголошення для них є нецікавими і не спонукають для кліку. Потрібно адаптовуватись і під цю групу юзерів, аби покрити усі вікові категорії. Проте водночас можна свідчити про результативність оголешень для юнаків та літніх людей, оскільки при меншій к-сті показів показник CTR 

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(age_group_data['Impressions'], age_group_data['CTR'], c='skyblue', marker='o', edgecolors='lightcoral')
plt.xlabel('Total Impressions')
plt.ylabel('Mean CTR')
plt.title('Середній CTR за загальною кількістю показів для кожної вікової групи')

# Annotate the points with age group labels
for i, age_group in enumerate(age_group_data['age_group']):
    plt.annotate(age_group, (age_group_data['Impressions'][i], age_group_data['CTR'][i]), textcoords="offset points", xytext=(0, 10), ha='center')


plt.locator_params(axis='x', nbins=50)
plt.xticks(rotation=45)

# Show the plot
plt.grid(True)
plt.show()

In [ ]:
men_over_18 = filtered_data[(filtered_data['Gender']==1) & (filtered_data['Age']>18)]
women_over_18 = filtered_data[(filtered_data['Gender'] == 0) & (filtered_data['Age'] > 18)]


Порівняємо покази для чоловіків і жінок віком старше 18 років

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x=['Male', 'Female'], y=[men_over_18['Impressions'].sum(),women_over_18['Impressions'].sum()])
plt.xlabel('Gender')
plt.ylabel('Impression')
plt.title('Порівняння к-сті показів між чоловіками та жінками старше 18 років')
plt.show()

Порівняємо к-сть кліків для чоловіків і жінок віком старше 18 років

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(x=['Male', 'Female'], y=[men_over_18['Clicks'].sum(),women_over_18['Clicks'].sum()])
plt.xlabel('Gender')
plt.ylabel('Clicks')
plt.title('Порівняння к-сті переходів між чоловіками та жінками старше 18 років')
plt.show()

Як бачимо жінки старше 18 років клікаю частіше ніж чоловіки, хоч жінки мають меншу к-сть показів. Щоб підтвердити дану інформацію, знайдемо CTR для кожної категорії і відобразимо на діаграмі 

In [ ]:
men_over_18['CTR'] = men_over_18['Clicks'] / men_over_18['Impressions']

women_over_18['CTR'] = women_over_18['Clicks'] / women_over_18['Impressions']

plt.figure(figsize=(8, 6))
sns.barplot(x=['Male', 'Female'], y=[men_over_18['CTR'].mean(),women_over_18['CTR'].mean()])

plt.xlabel('Gender')
plt.ylabel('Click-Through Rate (CTR)')

plt.title('Порівняння CTR між чоловіками та жінками старше 18 років')

plt.show()

In [ ]:
print(({"men_over_18_CTR": men_over_18['CTR'].mean()}))
print(({"women_over_18_CTR": women_over_18['CTR'].mean()}))

Проаналізуємо к-сть переходів авторизованих та неавторизованих юзерів.

In [ ]:
auth = user_data[user_data['Signed_In'] == 1]
non_auth = user_data[user_data['Signed_In'] == 0]

print({"auth":auth['Clicks'].sum(),"non_auth":non_auth['Clicks'].sum()})

sns.barplot(x=['Auth', 'Non-auth'], y=[auth['Clicks'].sum(), non_auth['Clicks'].sum()])
plt.xlabel('User Status')
plt.ylabel('Clicks')
plt.title('Порівняння к-сті кліків між авторизованими та неавторизованими користувачами')
plt.show()

Проаналізуємо к-сть показів авторизованих та неавторизованих юзерів.

In [ ]:
print({"auth":auth['Impressions'].sum(),"non_auth":non_auth['Impressions'].sum()})

sns.barplot(x=['Auth', 'Non-auth'], y=[auth['Impressions'].sum(), non_auth['Impressions'].sum()])
plt.xlabel('User Status')
plt.ylabel('Impressions')
plt.title('Порівняння к-сті показів між авторизованими та неавторизованими користувачами')
plt.show()

З вищенаведених візуалізацій можемо побачити, що к-сть показів для неавторизованих користувачів значно менша ніж для авторизованих, проте останні менше клікають на оголошення, тобто можна зробити висновок, що показник CTR для неавторизованих буде вищим, провізуалізуємо дане твердження.

In [ ]:
auth['CTR'] = auth['Clicks'] / auth['Impressions']
non_auth['CTR'] = non_auth['Clicks'] / non_auth['Impressions']

print({"auth_CTR":round(auth['CTR'].mean(),5),"non_auth_CTR":round(non_auth['CTR'].mean(),5)})

plt.figure(figsize=(8, 6))
sns.barplot(x=['Auth', 'Non-Auth'], y=[auth['CTR'].mean(),non_auth['CTR'].mean()])

plt.xlabel('User Status')
plt.ylabel('Click-Through Rate (CTR)')

plt.title('Порівняння CTR між авторизованими та неавторизованими')

plt.show()

З візуаліції можем побачити, що показник CTR для неавторизованих більше удвічі, що є дуже великою перевагою над авторизованими

Знайдемо квантилі, середнє значення, медіану, дисперсію та максимальне значення показів, кліків та CTR за віковими групами, за гендером та за статусом авторизації користувача

Для вікових груп:

In [ ]:
age_group_quantiles = filtered_data.groupby('age_group').agg(Impressions_0_25=('Impressions', lambda x: x.quantile(0.25)),
    Impressions_0_5=('Impressions', lambda x: x.quantile(0.5)),
    Impressions_0_75=('Impressions', lambda x: x.quantile(0.75)),
    CTR_0_25=('CTR', lambda x: x.quantile(0.25)),
    CTR_0_5=('CTR', lambda x: x.quantile(0.5)),
    CTR_0_75=('CTR', lambda x: x.quantile(0.75)),
    Clicks_0_25=('Clicks', lambda x: x.quantile(0.25)),
    Clicks_0_5=('Clicks', lambda x: x.quantile(0.5)),
    Clicks_0_75=('Clicks', lambda x: x.quantile(0.75)))
age_group_mean = filtered_data.groupby('age_group').agg(Mean_Impressions=('Impressions','mean'),Mean_CTR=('CTR','mean'),Mean_Clicks=('Clicks','mean'))
age_group_max = filtered_data.groupby('age_group').agg(Max_Impressions=('Impressions','max'),Max_CTR=('CTR','max'),Max_Clicks=('Clicks','max'))
age_group_median = filtered_data.groupby('age_group').agg(Median_Impressions=('Impressions','median'),Median_CTR=('CTR','median'),Median_Clicks=('Clicks','median'))
age_group_variance = filtered_data.groupby('age_group').agg(Variance_Impressions=('Impressions','var'),Variance_CTR=('CTR','var'),Variance_Clicks=('Clicks','var'))
age_group_df = pd.concat([age_group_quantiles,age_group_mean,age_group_variance,age_group_median,age_group_max],axis=1)
age_group_mean


In [ ]:
age_group_df

Звідси можемо побачити, що квантили, медіани співпадають у всіх вікових групах, максмальні значення майже однакові, дисперсія є невеликою, тому що для даних середніх значень дана дисперсія могла бути кардильно іншою, а вона приблизно така сама

In [ ]:
user_data['CTR']=user_data['Clicks']/user_data['Impressions'];

gender_quantiles = user_data.groupby('Gender').agg(Impressions_0_25=('Impressions', lambda x: x.quantile(0.25)),
    Impressions_0_5=('Impressions', lambda x: x.quantile(0.5)),
    Impressions_0_75=('Impressions', lambda x: x.quantile(0.75)),
    CTR_0_25=('CTR', lambda x: x.quantile(0.25)),
    CTR_0_5=('CTR', lambda x: x.quantile(0.5)),
    CTR_0_75=('CTR', lambda x: x.quantile(0.75)),
    Clicks_0_25=('Clicks', lambda x: x.quantile(0.25)),
    Clicks_0_5=('Clicks', lambda x: x.quantile(0.5)),
    Clicks_0_75=('Clicks', lambda x: x.quantile(0.75)))

gender_mean = user_data.groupby('Gender').agg(Mean_Impressions=('Impressions','mean'),Mean_CTR=('CTR','mean'),Mean_Clicks=('Clicks','mean'))
gender_max = user_data.groupby('Gender').agg(Max_Impressions=('Impressions','max'),Max_CTR=('CTR','max'),Max_Clicks=('Clicks','max'))
gender_median = user_data.groupby('Gender').agg(Median_Impressions=('Impressions','median'),Median_CTR=('CTR','median'),Median_Clicks=('Clicks','median'))
gender_variance = user_data.groupby('Gender').agg(Variance_Impressions=('Impressions','var'),Variance_CTR=('CTR','var'),Variance_Clicks=('Clicks','var'))

gender_df = pd.concat([gender_quantiles,gender_mean,gender_variance,gender_max,gender_median],axis=1)
gender_mean

In [ ]:
gender_df

Подібною до ситуації з віковими групами є ситуація з гендерами (квантили, медіани співпадають для всіх гендерів, максмальні значення майже однакові, дисперсія є невеликою,)

In [ ]:
auth_quantiles = user_data.groupby('Signed_In').agg(Impressions_0_25=('Impressions', lambda x: x.quantile(0.25)),
    Impressions_0_5=('Impressions', lambda x: x.quantile(0.5)),
    Impressions_0_75=('Impressions', lambda x: x.quantile(0.75)),
    CTR_0_25=('CTR', lambda x: x.quantile(0.25)),
    CTR_0_5=('CTR', lambda x: x.quantile(0.5)),
    CTR_0_75=('CTR', lambda x: x.quantile(0.75)),
    Clicks_0_25=('Clicks', lambda x: x.quantile(0.25)),
    Clicks_0_5=('Clicks', lambda x: x.quantile(0.5)),
    Clicks_0_75=('Clicks', lambda x: x.quantile(0.75)))
auth_mean = user_data.groupby('Signed_In').agg(Mean_Impressions=('Impressions','mean'),Mean_CTR=('CTR','mean'),Mean_Clicks=('Clicks','mean'))
auth_max = user_data.groupby('Signed_In').agg(Max_Impressions=('Impressions','max'),Max_CTR=('CTR','max'),Max_Clicks=('Clicks','max'))
auth_median = user_data.groupby('Signed_In').agg(Median_Impressions=('Impressions','median'),Median_CTR=('CTR','median'),Median_Clicks=('Clicks','median'))
auth_variance = user_data.groupby('Signed_In').agg(Variance_Impressions=('Impressions','var'),Variance_CTR=('CTR','var'),Variance_Clicks=('Clicks','var'))

auth_df = pd.concat([auth_quantiles,auth_mean,auth_variance,auth_median,auth_max],axis=1)
auth_mean

In [ ]:
auth_df


Статус авторизації користувача: квантили, медіани співпадають для всіх гендерів, максмальні значення майже однакові, дисперсія є невеликою,

Висновок:
У ході проведення дослідження було виявлено, що найбільш зацікавленою оголошеннями аудиторією є вікова група <18 та 54+. При меншій к-сті показів ніж у групи 18-54, було досліджено, що люди у віці менше 18 років та більше 54 переходять за оголошеннями 
частіше. Якщо дивитися на дану ситуацію з точки зору гендерів, то можна зробити висновок, що хоч жінки мають меншу к-сть показів, але вони переходять за оголошеннями частіше ніж чоловіки. Ситуація з авторизованими користувачами та неавторизованими така: неавторизована авдиторія більш активна, а ніж користувачі, що авторизувались - при меншій кількості показів, вони мають більшу кількість переходів та відповідно більший майже удвічі, показник CTR. Також згідно нашого дослідження, ситуація з метриками склалась ідентична у для всіх категорій дослідження (вікові групи, гендери, користувачі за статусом авторизації). У ході проведення дослідження було сформовано єдиний дата фрейм для кожної категорії, де можна отримати інформацію для кожної з метрик.